In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [ ]:
import sys
sys.getdefaultencoding()

Open the questionnaire file (word document version of the questionnaire saved as html):

In [ ]:
questionnaire_file = open("FF_f2_Questionnaire.htm", "r") 

In [ ]:
soup=BeautifulSoup(questionnaire_file, "html.parser")

In [ ]:
ps=[p for p in soup.find_all('p')]

In [ ]:
ps[164:168]

Opening the excel file with all the questions for p6, using the pandas python library:

In [ ]:
df=pd.read_excel("FFCf2.xlsx") #this is an excel file with all the questions for p6

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
reg = re.compile(r'(?P<section>[A-Za-z]*\d*)(?P<rest>.*)')

def format_q_num(name):
    """takes in the question numbers in the name column and outputs them in the format
    they appear in the text"""
    name=name.lstrip('f2')
    name=reg.search(name)
    section=name.group('section').upper()
    rest=name.group('rest').upper()
    return section+rest

In [ ]:
df['question_number']=df.name.apply(lambda x: format_q_num(x))

In [ ]:
df['multiple_answers']=df.question_number.apply(lambda x: 1 if re.match('[A-Z]\d*[A-Z]\d*',x) else 0) 
#adds a column where 1 indicates that it is one of the questions with multiple entries in the data frame
#I think these are mostly (only?) the "select all that apply" questions

In [ ]:
df

In [ ]:
def find_question(q_number):
    """finds where the question number appears in the text - only if it only appears once"""
    ps_text = []
    ps_idx = []
    ps_startswith = []
    for idx, p in enumerate(ps):
        if re.match('[A-Z]\d*[A-Z]',q_number):
            if q_number in p.get_text():
                if p.get_text().lstrip().startswith(q_number+'.'):
                    ps_text.append(p.get_text().lstrip().split('.')[1])
                    ps_idx.append(idx)
                else:
                    ps_text.append(p.get_text().lstrip().split('.')[0])
                    ps_idx.append(idx)
        else:
            if q_number+"." in p.get_text():
                if p.get_text().lstrip().startswith(q_number+"."):
                    ps_text.append(p.get_text().lstrip().split('.')[1])
                    ps_idx.append(idx)
    if len(ps_idx)!=1:
        return np.nan, np.nan
    else:
        return ps_text[0].replace("\n"," ").strip(), ps_idx[0]

In [ ]:
s = (find_question('A2'))
s

In [ ]:
def find_probe(q_number):
    probe = re.findall('[A-Z]{,1}\d{,2}', q_number)[0]
    return find_question(probe)[0]

In [ ]:
find_probe('B5A')

In [ ]:
#df.drop(['probe'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
for i in range(len(df.question_number)):
    q_number = df.loc[i,'question_number']
    if re.match('^[A-Z]{,1}\d{,2}', q_number):
        df.loc[i,'probe'] = find_probe(q_number)
    else:
        df.loc[i,'probe'] = "None"

In [ ]:
df

In [ ]:
df['ps_text'], df['ps_idx'] = zip(*df['question_number'].map(find_question))

In [ ]:
df[24:29]

In [ ]:
np.isnan(df.ps_idx).sum() #number of questions where the question number doesn't appear or appears more than once

In [ ]:
print(df.ps_text[16])
print(df.ps_idx[16])

In [ ]:
#get index of <p> tag for the next question so we can split the questions (can't just shift because some questions have more than one answer or missing values)
all_idx=np.array(df.ps_idx)
all_idx=all_idx[~np.isnan(all_idx)]
all_idx=np.append(all_idx, [len(ps)])
df['ps_idx_next']=df.ps_idx.apply(lambda x: all_idx[all_idx > x].min() if ~np.isnan(x) else np.nan)

In [ ]:
df[42:43]

In [ ]:
a3=ps[105:110]
a3

In [ ]:
consequitivedots = re.compile(r'\.{3,}')

In [ ]:
combined = [(p.find_previous('p').get_text() + p.get_text()) for p in a3]


In [ ]:
text = ''
j = 0
for p in a3:
    print(j)
    print(p.get_text())
    text = text + ' ' + p.find_previous('p').get_text()
    j += 1

In [ ]:
print(text)

test if lowercase letter, if so grab previous p as well.

In [ ]:
answers = [p for p in a3 if len(p.find_all(string = consequitivedots))>0]

In [ ]:
type(answers[0])

In [ ]:
print(answers[0].prettify())

In [ ]:
test_answers = [p for p in a3 if len(p.find_all(string = consequitivedots))>0]
for ans in test_answers:
    if re.match('^[a-z]{1}', ans.get_text().split('.')[0]):
        print(ans.find_previous('p').get_text() + ans.get_text().split('.')[0])
        print(ans.get_text().split("(")[0].split('.')[-1])
    else:
        print(ans.get_text().split('.')[0])
        print(ans.get_text().split('.')[-1])

In [ ]:
def clean_text(string):
    return string.replace("\n"," ").strip()

def get_pairs(p):
    if re.match('^[a-z]{1}', p.get_text().split('..')[0]):
        if re.match('[()]{1}', p.get_text().split('..')[-1]):
            pairs = [p.find_previous('p').get_text() + ' ' + p.get_text().split('..')[0], p.get_text().split("(")[0].split('..')[-1]]
        else:
            pairs = [p.find_previous('p').get_text() + ' ' + p.get_text().split('..')[0], p.get_text().split('..')[-1]]
    else:
        if re.match('[()]{1}', p.get_text().split('.')[-1]):
            pairs = [p.get_text().split('..')[0], p.get_text().split("(")[0].split('..')[-1] ]
        else:
            pairs = [p.get_text().split('..')[0], p.get_text().split('..')[-1]]
    pairs = [clean_text(text) for text in pairs]
    return pairs
        
def get_answers(start_idx,stop_idx):
    if np.isnan(start_idx) or np.isnan(stop_idx):
        return ([],0)
    else:
        answers = [p for p in ps[int(start_idx):int(stop_idx)] if len(p.find_all(string = consequitivedots))>0]
        answers = [get_pairs(a) for a in answers]
        return(answers, len(answers)) 
        

In [ ]:
df['answers'], df['n_answers'] = zip(*map(get_answers, df['ps_idx'], df['ps_idx_next']))

In [ ]:
(df['answers'][19])

In [ ]:
x = 0
while x < len(df.answers):
    ans = df.answers[x]
    if len(ans) > 0:
        for y in range(len(ans)):
            df.loc[x,'val{0}'.format(y)] = ans[y][1] #create new 'val(y)' column
            df.loc[x,'lab{0}'.format(y)] = ans[y][0] #create new 'lab(y)' column
    x += 1

In [ ]:
def clean_answers(col):
    col.str.replace('.','',1)

In [ ]:
values = []
for i in range(0,18):
    values = values + ['val{0}'.format(i)]
    i += 1
for v in values:
    df[v] = df[v].map(lambda x: str(x))
    df[v] = df[v].map(lambda x: x.lstrip('.').strip())

In [ ]:
df['val0']

In [ ]:
i = 0
while i < len(df.ps_text):
    if df.multiple_answers[i] == 1:
        df.loc[i,'val0'] = '1'
        df.loc[i,'lab0'] = 'YES'
        df.loc[i,'val1'] = '2'
        df.loc[i,'lab1'] = 'NO'
    i += 1

In [ ]:
df.drop(['answers','n_answers','question_number','ps_idx','ps_idx_next'],axis=1,inplace=True)

In [ ]:
df.head(20)

In [ ]:
df.to_csv(path_or_buf = "f1Qtext.csv")